In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
os.getcwd()
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
# #         print(dirname)
os.chdir('/kaggle/input/handwriting-bci/handwritingBCIData/Datasets')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Loading the data

In [5]:
os.getcwd()
os.listdir()
fnames = []
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        fnames.append(os.path.join(dirname, filename))
fnames = [fname for fname in fnames if fname.find('singleLetters')!= -1]
fnames

['./t5.2020.01.13/singleLetters.mat',
 './t5.2020.01.08/singleLetters.mat',
 './t5.2019.12.09/singleLetters.mat',
 './t5.2019.11.25/singleLetters.mat',
 './t5.2020.01.15/singleLetters.mat',
 './t5.2019.12.20/singleLetters.mat',
 './t5.2019.12.11/singleLetters.mat',
 './t5.2020.01.06/singleLetters.mat',
 './t5.2019.05.08/singleLetters.mat',
 './t5.2019.12.18/singleLetters.mat']

In [36]:
from scipy.io import loadmat
dataset = [loadmat(fname) for fname in fnames]

In [37]:
# functionn to prepare the data
def get_session_data(dat):
    data = []
    labels = []
    for key in dat.keys():
        if key.find('neuralActivityCube') != -1 and key.find('doNothing') == -1:
            label = key.split('_')[1]
            ch_label = np.array([label] * dat[key].shape[0])
            labels.append(ch_label)
            data.append(dat[key])
    data = np.concatenate(data, axis=0)
    labels = np.concatenate(labels, axis=0)
    return data, labels
def get_data(dataset):
    data = []
    labels =[]
    for session in dataset:
        d, l = get_session_data(session)
        data.append(d)
        labels.append(l)
    data = np.concatenate(data, axis=0)
    labels = np.concatenate(labels, axis=0)
    return data, labels

In [38]:
data, labels = get_data(dataset)
data.shape, labels.shape

((3627, 201, 192), (3627,))

In [39]:
import random
import torch

def set_seed(seed=None, seed_torch=True):
    '''Adapted from NMA-DL tutorials'''
    if seed is None:
        seed = np.random.choice(2 ** 32)
    random.seed(seed)
    np.random.seed(seed)
    if seed_torch:
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True

    print(f'Random seed {seed} has been set.')


# In case that `DataLoader` is used
def seed_worker(worker_id):
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

SEED = 2021
set_seed(seed=SEED)

Random seed 2021 has been set.


In [40]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
Y = enc.fit_transform(labels.reshape(-1, 1))

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, Y, shuffle=True)

In [42]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader


In [43]:
from torch.utils.data import DataLoader, TensorDataset


X_train = torch.tensor(X_train, dtype=torch.double).reshape(-1, 1, 201, 192)
X_test = torch.tensor(X_test, dtype=torch.double).reshape(-1, 1, 201, 192)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

train = TensorDataset(X_train, y_train)
test = TensorDataset(X_test, y_test)
train_loader = DataLoader(train)
test_loader = DataLoader(test)

In [45]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.pool1 = nn.MaxPool2d(kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.pool2 = nn.MaxPool2d(kernel_size=3)
        self.fc1 = nn.Linear(26880, 1024)
        self.fc2 = nn.Linear(1024, 31)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = torch.flatten(x,1)
#         print(x.shape)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return(x)

In [46]:
device = 'cuda'
def train():
    net.train()

    for d in train_loader:  # Iterate in batches over the training dataset.
        inputs, labels = d
        inputs = inputs.to(device).double()
        labels = labels.to(device).long()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
def test(data_loader):
    net.eval()
    correct = 0
    total = 0
    for data in data_loader:
      inputs, labels = data
      inputs = inputs.to(device).double()
      labels = labels.to(device).long()

      outputs = net(inputs)
      _, predicted = torch.max(outputs, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    return total, acc


In [ ]:
net = ConvNet().double().to('cuda')
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-3)
num_epochs = 50

tr_acc = []
te_acc = []
for epoch in range(num_epochs):
    train()
    _, train_acc = test(train_loader)
    _, test_acc = test(test_loader)
    print(f'Training Accuracy: {train_acc}, Test Accuracy: {test_acc}')
    tr_acc.append(train_acc)
    te_acc.append(test_acc)

Training Accuracy: 6.727941176470588, Test Accuracy: 4.74090407938258
Training Accuracy: 5.477941176470588, Test Accuracy: 3.197353914002205
Training Accuracy: 9.411764705882353, Test Accuracy: 5.2921719955898565
Training Accuracy: 18.49264705882353, Test Accuracy: 12.127894156560089
Training Accuracy: 34.375, Test Accuracy: 28.66593164277839
Training Accuracy: 61.3235294117647, Test Accuracy: 48.4013230429989
Training Accuracy: 74.15441176470588, Test Accuracy: 54.13450937155458
Training Accuracy: 80.91911764705883, Test Accuracy: 56.008820286659315
Training Accuracy: 79.8529411764706, Test Accuracy: 53.80374862183021
